In [ ]:
!pip install unsloth transformers datasets trl torch pandas numpy
import torch
from unsloth import FastLanguageModel
from datasets import Dataset
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForLanguageModeling
import pandas as pd
import numpy as np

Testing on randomly generated price_stream and structure the data

In [ ]:
# Simulate price stream data (replace with actual data source)
price_stream = pd.DataFrame({
    "timestamp": pd.date_range(start="2023-01-01", periods=1000, freq="T"),  # 1-minute intervals
    "price": np.random.normal(loc=100, scale=2, size=1000).cumsum()  # Simulated price movement
})

def create_stream_windows(data, window_size=50):
    windows = []
    for i in range(len(data) - window_size):
        window = data.iloc[i:i + window_size][["timestamp", "price"]].to_dict(orient="records")
        windows.append(window)
    return windows

price_windows = create_stream_windows(price_stream)

Generate Training Example with reasoning

In [ ]:
def generate_reasoned_strategy(window, risk, style):
    prices = [tick["price"] for tick in window]
    timestamps = [tick["timestamp"] for tick in window]
    
    # Simple mathematical reasoning: detect a trend based on price change
    price_change = prices[-1] - prices[0]
    avg_price = np.mean(prices)
    volatility = np.std(prices)
    
    # Timing logic based on style
    lookback = 5 if style == "intraday" else 20 if style == "MFT" else 50
    
    # Construct MQL5 strategy with math and timing
    strategy_code = f"""
// {style} strategy with {risk}% risk
input double RiskPercent = {risk};
input int Lookback = {lookback};

void OnTick()
{{
    // Get price stream
    double currentPrice = Close[0];
    double pastPrice = Close[Lookback];
    double priceChange = currentPrice - pastPrice;
    double volatility = iStdDev(NULL, 0, Lookback, 0, MODE_SMA, PRICE_CLOSE, 0);
    double lotSize = AccountBalance() * RiskPercent / 100 / (volatility * 1000);
    
    // Mathematical and logical reasoning
    if (priceChange > volatility && currentPrice > {avg_price})
    {{
        // Buy when price increases significantly
        OrderSend(Symbol(), OP_BUY, lotSize, Ask, 3, 0, 0);
    }}
    else if (priceChange < -volatility && currentPrice < {avg_price})
    {{
        // Sell when price decreases significantly
        OrderSend(Symbol(), OP_SELL, lotSize, Bid, 3, 0, 0);
    }}
}}
"""
    return strategy_code

# Create dataset
dataset = []
for window in price_windows[:100]:  # Subset for demonstration
    risk = np.random.uniform(1, 5)
    style = np.random.choice(["intraday", "MFT", "investment"])
    strategy_code = generate_reasoned_strategy(window, risk, style)
    dataset.append({
        "window": window,
        "risk": risk,
        "style": style,
        "code": strategy_code
    })

Format the dataset

In [ ]:
def format_example(example):
    price_text = " ".join([f"{tick['price']}" for tick in example["window"]])
    prompt = f"Price stream: {price_text}\nRisk: {example['risk']}%\nStyle: {example['style']}\n\nStrategy Code:\n"
    return {"text": prompt + example["code"]}

dataset = Dataset.from_list(dataset)
dataset = dataset.map(format_example)

Load and Configure the model

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/DeepSeek-R1-Distill-Llama-8B",
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing=True,
)

Preprocess the data

In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

Fine Tune the Model

In [ ]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=512,
    data_collator=data_collator,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=60,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
    ),
)

trainer.train()

model.save_pretrained("fine_tuned_model")
tokenizer.save_pretrained("fine_tuned_model")

Generate Strategies

In [ ]:
def generate_strategy(price_window, risk, style):
    price_text = " ".join([f"{tick['price']}" for tick in price_window])
    prompt = f"Price stream: {price_text}\nRisk: {risk}%\nStyle: {style}\n\nStrategy Code:\n"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        temperature=0.6,
        do_sample=True
    )
    generated_sequence = tokenizer.decode(outputs[0], skip_special_tokens=True)
    generated_code = generated_sequence[len(prompt):].strip()
    return generated_code

# Example usage
sample_window = price_windows[0]  # Replace with actual price stream window
risk_percentage = 2
trading_style = "intraday"
print("Generated Strategy:")
print(generate_strategy(sample_window, risk_percentage, trading_style))

*Additional Considerations

- Real Data: Replace the simulated price stream with actual tick data from a reliable source for better results.
- Enhanced Reasoning: To improve the model’s ability to innovate, consider reinforcement learning after initial supervised fine-tuning, rewarding strategies that optimize performance metrics.
- Scalability: Increase the dataset size (e.g., thousands of windows) and training steps for better generalization.
- Limitations: The DeepSeek R1 Distill Llama 8B is a language model, not natively designed for time-series tasks. If results are suboptimal, explore hybrid approaches with time-series models (e.g., LSTMs) to preprocess data before feeding it to the language model.

This solution provides a complete framework to fine-tune a model on price stream data, enabling it to generate mathematically reasoned trading strategies in MQL5, tailored to your specifications. Let me know if you’d like to refine any part further!